# Creating an unpivot function for MySQL

MySQL does not contain a feature that allows us to unpivot a table easily. The goal of the function below will be to:  
*    Extract the whole table needed to build the unpivoted version  
*    Create a temporary table, suffixed \_tidy, that will contain the final data  
*    Tidy the data using pandas, within python itself, using tidy data principles  
*    Re-upload the table as a new, tidied up table within the database.  

**Important note:** Using this function will require the MySQLdb module to be installed on your computer.   
More information on how to install at the link below:  
https://pypi.python.org/pypi/MySQL-python/1.2.5  
http://mysql-python.sourceforge.net/MySQLdb.html


In [ ]:
def unpivot(host, user, pwd, db1, table, name_col = 'VARIABLE', port = 3306, pvt_cols = [], column_index = []):
    
    #Import MySQLdb and Pandas
    import MySQLdb
    import pandas as pd
    from sqlalchemy import create_engine
    
    #Extract values from table to unpivot
    db=MySQLdb.connect(host=host,user=user,passwd=pwd,db=db1,port=port)
    engine = create_engine('mysql://{0}:{1}@{2}:{3}'.format(user, pwd, host, port))
    engine.execute("USE "+db1)
    c = db.cursor()
    c.execute("SELECT * FROM " + table)
    c.execute("DROP TABLE IF EXISTS " +table+"_TIDY")
    data = list(c.fetchall())
    header_names = [i[0] for i in c.description]
    
    #constitute pd dataframe from data + headers
    df = pd.DataFrame(data,columns=header_names)
    
    id_vars = []
    for i in range(len(header_names)):
        if header_names[i] in pvt_cols:
            1==1
        else:
            id_vars.append(header_names[i])
    
    #Proceed to unpivot 
    df = pd.melt(df,id_vars=id_vars,value_vars=pvt_cols,var_name=name_col)
    
    df.to_sql(name=table+'_TIDY', con=engine, if_exists = 'replace')
        
    